In [ ]:
import os
LAB_ROOT = "."   # stay in current folder
os.makedirs("data", exist_ok=True)
os.makedirs("outputs", exist_ok=True)
import os; print("Working in:", os.getcwd()); print("Folders ready: data/, outputs/")

In [ ]:
text = """Once upon a time, a curious student built a small machine that could learn.
The machine read thousands of pages, numbers, and songs, trying to make sense of the world.
Every day it grew a little smarter and a little kinder.
It learned to help people cook, drive, and dream.
But one day the student asked, “Will you ever replace me?”
The machine smiled in silence and wrote on the screen:
  I was never meant to replace you.
  I was meant to remind you how powerful you are when you learn.

From that day, the student and the machine worked together,
creating art, solving problems, and teaching others to imagine a better world."""
with open("data/ai_future_story.txt", "w", encoding="utf-8") as f:
    f.write(text)
print("Created data/ai_future_story.txt")

In [ ]:
# This mirrors the classic Beam wordcount sample: run(argv=None), --input/--output flags, named transforms.

import argparse
import os
import re
import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions

class SplitWords(beam.DoFn):
    def process(self, element):
        line = element.lower()
        line = re.sub(r"[^a-z0-9\s']", " ", line)  # keep letters, digits, space, apostrophes
        for w in line.split():
            if w:
                yield w

def format_result(word_count):
    word, count = word_count
    return f"{word}\t{count}"

def run(argv=None):
    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--input",
        default="data/kinglear.txt",   # same default as the sample; we’ll override below
        help="Input path (file or glob), e.g., data/ai_future_story.txt or data/*.txt",
    )
    parser.add_argument(
        "--output",
        default="outputs/wordcount_output",
        help="Output prefix (Beam adds shard suffixes), e.g., outputs/wordcount_ai_future",
    )
    known_args, pipeline_args = parser.parse_known_args(argv)

    os.makedirs(os.path.dirname(known_args.output) or ".", exist_ok=True)

    pipeline_options = PipelineOptions(pipeline_args)  # add runner args here if you need
    with beam.Pipeline(options=pipeline_options) as p:
        lines = p | "Read" >> beam.io.ReadFromText(known_args.input)
        words = lines | "Split" >> beam.ParDo(SplitWords())
        pairs = words | "PairWithOne" >> beam.Map(lambda w: (w, 1))
        counts = pairs | "CountPerWord" >> beam.CombinePerKey(sum)
        _ = (counts
             | "Format" >> beam.Map(format_result)
             | "Write" >> beam.io.WriteToText(known_args.output))


In [ ]:
# Process the file we just created

argv = [
    "--input",  "data/ai_future_story.txt",
    "--output", "outputs/wordcount_ai_future",
    # You can also pass runner options here, e.g. "--runner", "DirectRunner"
]
run(argv)
print("Done.")


In [ ]:
import glob, os

parts = sorted(glob.glob("outputs/wordcount_ai_future*"))
print("Output shards:", parts)

for p in parts:
    print(f"\n=== {os.path.basename(p)} ===")
    with open(p, "r", encoding="utf-8") as f:
        print(f.read(), end="")
